In [2]:
import sys
sys.path.append('../src')

import model_utils

#### Machine Learning Model Functions:

a. CNN():
This function creates a Convolutional Neural Network (CNN) model for image classification. It consists of a series of convolutional, pooling, and dense layers, along with activation functions, dropout, and normalization layers.

b. SVM():
This function creates a Support Vector Machine (SVM) model for image classification. It uses a kernel function to map the images into a high-dimensional feature space, where it finds the optimal hyperplane that separates the cancerous and non-cancerous cells or the different cell types.

c. Random Forest():
This function creates a Random Forest model for image classification. It builds multiple decision trees on randomly selected subsets of the data and features and combines their predictions to obtain a final classification.

#### Hyperparameter Tuning Functions:

a. grid_search():
This function performs a grid search over a range of hyperparameters for a given model and dataset, and returns the best combination of hyperparameters that yields the highest cross-validation accuracy.

b. random_search():
This function performs a random search over a range of hyperparameters for a given model and dataset, and returns the best combination of hyperparameters that yields the highest cross-validation accuracy.

c. plot_hyperparameters():
This function plots the results of a hyperparameter search as a heatmap or scatter plot, showing the correlation between different hyperparameters and their effect on the model performance.

These are just some examples of Python functions that you can use for task 1.2. You may need to customize or combine them based on your specific requirements and data.